In [83]:
import os
import json
import pandas as pd
from pandas import json_normalize
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.interpolate import make_interp_spline
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

from scipy.stats import randint

In [84]:
#This document is for frequencies and probablility

In [117]:
dataset = pd.read_csv('csv/flattened_matches.csv')
df = dataset[dataset['queue_id']== 1100].copy() #ranked data only
df = df[df['placement']<= 4].copy() #ranked data only
summonUnits = ['unit_TFT14_AnnieTibbers', 'unit_TFT14_SummonLevel2', 'unit_TFT14_SummonLevel4', 'unit_TFT14_Summon_Turret']
df.drop(summonUnits,axis='columns',inplace=True)

In [ ]:
column_key = df.keys()
column_key = [item for item in column_key if ( "unit_" in item and "_item" not in item)]

unit_key = column_key.copy()
column_key.insert (0,'id')


precondition_probablility_unit = pd.DataFrame(columns=column_key)
precondition_probablility_unit['id'] = unit_key

print ( precondition_probablility_unit.shape )
# Loop through each column in search_key
for u in tqdm(search_key, desc="Looping over search_key"):
    # Loop through each row in precondition_probablility_unit
    for index, row in precondition_probablility_unit.iterrows():
        # Check if the value in df[row['id']] for the current row is not 0
        non_zero_count = 0
        hasRows = df[df[row['id']] != 0]
        non_zero_count = ((hasRows[u] != 0) & (hasRows[row['id']] != 0)).sum() / hasRows.shape[0]
        
        # Update the precondition_probablility_unit DataFrame
        precondition_probablility_unit.at[index, u] = non_zero_count

(60, 61)


Looping over search_key:  33%|███████████████████                                      | 20/60 [00:16<00:33,  1.19it/s]

In [ ]:
precondition_probablility_unit

In [ ]:
import plotly.express as px
import pandas as pd
import plotly.io as pio

# Extract the ID column for row labels
id_column = precondition_probablility_unit['id']

# Drop the 'id' column from the data for the heatmap
precondition_probablility_unit_cleaned = precondition_probablility_unit.drop(columns=['id'])

# Ensure the data is numeric
numeric_data = precondition_probablility_unit_cleaned.apply(pd.to_numeric, errors='coerce')

# Create an interactive heatmap using plotly
fig = px.imshow(numeric_data,
                labels={'x': 'Search Key Columns', 'y': 'Given this unit', 'color': 'Probability'},
                x=numeric_data.columns,
                y=id_column,
                color_continuous_scale='YlGnBu')

# Update layout to remove annotations, show hover info, and adjust chart size
fig.update_traces(texttemplate='', hoverinfo='y+x+z')  # Remove annotations, show hover info

# Set the size of the chart (width and height)
fig.update_layout(
    title="Precondition Probability Heatmap",
    xaxis_title="Search Key Columns",
    yaxis_title="Given this unit",
    width=1500,  # Increase width
    height=1200   # Increase height
)

# Show the plot
#fig.show()
pio.show(fig)

In [ ]:
overall_probablility_unit = pd.DataFrame(columns=unit_key)
search_key = overall_probablility_unit.keys()

for u in search_key:
    overall_probablility_unit[u] = [(df[u] != 0).sum() / df.shape[0]]
    #print ( f"{(df[u] != 0).sum()} / {df.shape[0]} ")

In [ ]:
overall_probablility_unit

In [ ]:
plot_data = overall_probablility_unit.T
plot_data.columns = ['non_zero_ratio']
plot_data_sorted = plot_data.sort_values(by='non_zero_ratio', ascending=True)

# Plot
plot_data_sorted.plot(kind='bar', legend=False, figsize=(10, 5))
plt.title('Proportion of Unit appearing at Final Board')
plt.ylabel('Proportion')
plt.xlabel('Unit')
plt.ylim(0, 1)
plt.grid(axis='y')
plt.tight_layout()
plt.show()